In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Limpieza de datos. Apply. Expresiones regulares.

La limpieza es un paso necesario en todo proyecto de datos. 

Podemos resumir el proceso de limpieza de datos en las siguientes cinco tareas:

**1. Resolver problemas de formato y asignar tipos de datos correctos.**

Por ejemplo, cuando al pasar de CSV a Pandas una fecha no se importa correctamente como puede ser el caso de un campo fecha donde se importa 20090609231247 en lugar de 2009-06-09 23:12:47.

El formato en que se encuentran los datos determina qué operaciones pueden realizarse sobre ellos.

**2. Estandarizar categorías.**

Cuando los datos se recolectaron con un sisstema que no tiene valores tipificadaos, valores que representan la misma categoría pueden estar expresados de forma distinta. Por ejemplo: Arg, AR, Argentina

**3. Corregir valores erróneos.**

Por ejemplo: un valor numérico o inválido para describir el género; o una edad representada por un número negativo o mucho mayor que 100.

**4. Completar datos faltantes.**

Los datasets del mundo real suelen venir con datos faltantes que responden a información que se perdió o nunca se recolectó. Existen varias técnicas para completar datos faltantes. Al proceso de completar datos faltantes se lo llama "imputación".

**5. Organizar el dataset.**

Es importante estructurar las filas y columnas de la forma más conveniente. Para hacerlo se pueden aplicar las reglas del "tidy data".



## Dataset

El Met (Metropolitan Museum of Art) provee datasets de información de más de 420.000 piezas de arte en su colección.

https://github.com/metmuseum/openaccess/

Los problemas que presentan este dataset incluyen:
* Valores faltantes
* Posibles duplicaciones
* Campos con mezcla de campos de tipo numérico con cadenas de caracteres

En esta clase vamos a detectar algunos de los qué campos presentan problemas, y solucionarlos usando las herramientas que adquirimos en las prácticas guiadas.


## Ejercicio 1:

Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos cuántas filas y columnas tiene el DataFrame data

In [1]:
import pandas as pd

In [4]:
data_path="../Data/MetObjects_sample.csv"

data= pd.read_csv(data_path)
print(data.shape)
data.head(3)

(4743, 52)


,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
0,297513,Inst.1980.3.1,False,False,False,442985,Islamic Art,NaN,Illustrated single work,Illustrated Single Work,...,NaN,NaN,Codices,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Abstraction,http://vocab.getty.edu/page/aat/300056508
1,366196,175T49 R43,True,False,False,591826,The Libraries,NaN,NaN,Vita del Tintoretto,...,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q29385980,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
2,380983,WW.550,False,False,False,650203,Drawings and Prints,NaN,Print,Flowers and Dragonfly,...,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Dragonflies|Flowers,http://vocab.getty.edu/page/aat/300310476|http...


## Ejercicio 2: Formato y tipos de datos

Vamos a ver de qué tipo de datos es cada columna del DataFrame, y vamos a convertir o dar formato **a alguna de las columnas** que tienen tipo de datos incorrecto.

### 2.a Detectar las columnas que tienen tipo de datos incorrecto

In [12]:
data.isnull().sum()

data.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

### 2.b AccessionYear

Analizar la columna AccessionYear que fue leída como object, y debería ser int.

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos extraer el dato año de los valores no numéricos, y crear una nueva columna en el DataFrame de tipo int que se llame AccessionYearClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

Observación: si la columna tiene valores NaN no vamos a poder convertirla al tipo int. Una opción es reemplazar los valores NaN por algún entero antes de convertir (`fillna`). La otra opción es dejar la columna como tipo float (el tipo de NaN es float).


In [14]:
#display(data['AccessionYear'])

data['AccessionYear'].value_counts()

1963          220
1917          140
1874          126
1929           99
2011           98
             ... 
2005-02-15      1
1873            1
1892            1
2020-03-23      1
1986-12-29      1
Name: AccessionYear, Length: 142, dtype: int64

In [15]:
data['AccessionYear'].isnull().sum()

1000

In [16]:
import re
import numpy as np

## Ejercicio 3: Categorias - Valores erróneos

Miremos ahora el campo "Artist Gender"

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos definir como categorías válidas Male, Female y Unknown

Y crear una nueva columna en el DataFrame que se llame ArtistGenderClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

¿Podemos deducir cómo está representada la categoría Male en el dataset original?

Nota: La propuesta que hacemos para limpiar este campo no es del todo correcta, y vamos a ver por qué más adelante. Pero sirve como ejercicio.


## Ejercicio 4: Imputación

Vamos a analizar ahora los campos "Object Date", "Object Begin Date", "Object End Date"

**4.a ¿Cuántos valores nulos hay en "Object Date"? ¿Cuántos en "Object Begin Date"? ¿Cuántos en "Object End Date"?**


**4.b Usaremos los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

1) Vamos a crear una columna nueva ("Object Date 4b") donde copiamos todos los datos de Object Date (para no cambiar los valores originales y nos sirvan para el próximo ejercicio)

2) Vamos a rellenar la columna "Object Date 4b" con la estrategia que planteamos.

**4.c Usaremos aleatoriamente los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

Para eso definimos una función get_fill_value que recibe como parámetro una fila da data, y si el valor del campo "Object Date" es nulo devuelve aleatoriamente el valor del campo "Object Begin Date" u "Object End Date" de ese registro.

(En este ejercicio practicamos todo!)

Vamos a contar ahora cuántos registros rellenó con los valores de Object Begin Date y cuántos con Object End Date

## Nota: Organizar el dataset

Para llevar a cabo esta tarea necesitamos algunas herramientas que veremos en la segunda parte de Pandas.

En esa clase vamos a volver a este ejercicio y resolver este punto.

Los campos que vamos a analizar son "Artist Nationality" y "Artist Display Name"

In [ ]:
data["Artist Nationality"].value_counts()

In [ ]:
data["Artist Display Name"].value_counts() 